In [11]:
#####  台彩大樂透

import requests
from bs4 import BeautifulSoup as bs

url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

### 發 POST requests 取得頁面 source code
payload = {
    "__VIEWSTATE": "/w+v0tcu4EBEs5quMQlWYo9XcfrOkD+nUXBT5WffgSV9ukK9z9PaOBjuCrotOqmR3yhWSLF9EPkmUEgIH8RX2YXAR87ZaWNxJkKCyop5sHSfOOldToMT8jgumXUB4TraL7vDyzU2s1uCxcXrLMYIVrwq6btykjAGdtpbfaP1khci5CEmkyq7MLTO7mIPw6+1kmQOuQiQkWKUgOs/m1CyRyww/b/UVaXGuwJvKWNCfMBQd+QjfUqvbenlDKk4INDNtgRinIbqH6Fuu9QfEEfD/tjc4e0XsxKQw/2DZ1SiK9SJtnuxuwB5EahJI4YXI1ULosJaOfN4vbPx9DYB+smnpwX0embGaj3Hmulj24MtSP7VDR/S/VKHK8bqu37ljZGnXE7p7xmKxxC6O2EnBNlMWEWEVvAkDE4L8/zz/EWXPDnG6mKI6WGwA8MQ+/3yyV1fH2iE1z5hnNCSMAdccwZqR0Ifaz8pxmXIksGj7aAXYuYVRCVgr9UEpf9S5xBPN8hSdO1HA+tVzEC+SxSBaSoQfn4/jjgu6wBrMGisBXmfuuUGwtIVHsRDmWNWQb30qY6jZef1P7tox48c1msrS4oRZQXWCwav93yBLul+RqblEtcbrrHlcLL2ZiLg2LPG5VyaQw+eNTzqNyLcylcYsYAXzR3VQUWoD/VVcEF0d5MnUJp/8l3SZRS57lDOQTqfxDf4T0D+pqY5z/MQvkAhrJ+xTXNY6pk8W9O5nzIuL7GeIZWT8aULxhimp/T1Qxa7FIq1kPxdOIOSL+45sM3Ulbs1s+ESbIRN88Ogr3KLh6BJlu2zR7N+P2o37DoqzVJxbLeSkBO+w0BNQnJ1PtT+4cKQBdYPPXMEyYmgpEdkVrcnAzHiRfSkYjNumvsjZFtw2DRWTqHVEbKYdEyR21dPMyPacovlG1QfePZtXsKOI36syF0UpB0W4rvko5hD7jnUfb6aFswWLLnhoKs9qcTK0GIQwnYnaVibjWqD69/H+Va54Yh0RQyJUYtSxiAqE0jT/OLl4TaXo7xOgRxYA/+MIQWbgQI1nGv/1Wb/74ph1fBTyOcFhj5InEy4RDFu8ZZPKE5Rx0XSLKG3MR0spnlbFwWWha8Pm3yjst0UzIMQc30PkSoV8xUgX0KSPsYLzt1VJOKwxCOzDHbKnT66ZdukM56XdkF0nGQEcHR6QDjp1+qYgvpTVZXx8gAa1zDlfahJ1XIOT3DqOo+eSMiXViJy0CLOodI+BLSwcGSqcqXMxyhnt+Y7NsagoJS8UxkSBB6DLoMDkqKvbLxU6A+cUrWyXKp5XqlsPp0SI3lHoBW2PI0Y0gf5rEjFTzLhWSN/8B1Yuh/yzdsghU3mHLWAyriByTc9nI8zkGvTgfAh4a/AFC/PSjkPe/OvYDa5F0tA4/jI2qr3GxFxptfi9b/4qTwmERtQzb/BGOUNiIw9L80pmdk36SAK8ZG552c+HciWoXw4dXijksgKGfBtP3fuTrJz25FORGpANp5oajpmQKLkjNUgZu+2M4BxlJnpBNsAHJWcTpyqRPkvzIP6Fr+tDylMrCPbG92cEgr3s4Sq850X5H1qnP6fh+eqRMrzbINkmVW/eprs2NBPpCngxqZVhRoldt3k6hz89WWXPKUgOb5V1J/MwOpX6xoDuF3J7dNlp4qyqet57fxMyFzDyOLswvwLt0kNYMQ9XoxjhXWnVamowmwAIE04sKThWhqCxi+5CoUdi7xpdye08cryt3lE8Ig0ldgi+ew1JliKgRhz/psCk7k9Lyy3dGGenPFHe1jzd06vVggYgjuNbFg2IwpVWWSNU0j4WDORECl1ba2zFu6M0IAKQDEeoRNYskOe0S8eRqJWk7V//FIaenhb0orcnnInyqKYN5mr3/3IU8WGVRyTBUPpMMQHgwVtYv91ORE5klQ0LtLlzKvJtxC/S1KQYnrsBtHOGTwK6qtpTbhtmjyub2vFjchJNnbK59MpTuSVuVm4jYuVc0XOckKBIZw6FOI6szxZj1Ut8qrYRfdA13FWnZsdM4xxY+n9PMH3mwC4Q3aZ6b7A2y9s7RX1+QtEwNctL6eOuOMVRRYg/J0GioUdcigtdTKFkhUS3hUYDGLEC75oErVq6omLIuVTolekG/ChSr+1Cefa7rnvO4GQxIqg7rcIzQ8Rc41/7uO8Yp/i3OceNoJvmKVqZL91Du9d5xq+B3VxHWnsSXOyXHjsVw67v3zldACSRlimEYBDpKqC2r0kgnGM24Z3cDC53+oC7zjL410Y7jfSy/v5gdNFElg9etvqXaH4vsMJ5gxpn/+JXvmYqcLFOUYtpWZrfD4ImQCorKa4Yw4YOIr2j2iDi7J39lPSr3WKElH5FKt3vusPalQujpquOOCzrivkiSSzJUQ94YCY8JtItt9Cae0QHDpvWcnWEssPeBG6QvIxU11QtJxA8pJoexSA+kL32EcFkPbcRWlIPMYBvTijuXdr1O9ddU9DZ4lFWakVjyrqHL2hw5bJj4C1SwLAmpD39lTLXNtV6UiSkgXZ5V9CZyGjUch2FvKRitqR6DIjxr5BtaCJHrtLQ5VMWwiO/onTADWNt0EuZcRkTAcu9yp23LqjfzHU2OUYiu3nw2KhcipsPGqendw/xWL05CESKUzpghm1ISWU6JxZ+yqW9q4TpSm4iI5H8w+4KPUWCcSGt9nsgZNns/DXLw6td6dTigKx26uiqsurVfk0jW6d7PtQ8avgrGKKXVgfGEhPqV335y4oXCpAM6mBYPmE33qjD6cZG34lPn7qzCwlB6MHf/UHFwjvE2i7bQMPY1JUJTayLon1t7kmcBSF0uRSkFuDlRDlQ4HEkw9HrFMWUPxlT9cryuWaHa4E9cFuRnwbD7DdOCqfgKyoqhwHQLlI19/bQcoVBr4SWxWTfF1vWpXrZKLl96AAHlmpxIRr4Z+N64eeLxLg0jERAxGDlOFmfKix19q1jtI+oSNM4HSwpArzCyZu8L1Zh6s52+O6pi6L28gyNXnGilf1ZurTQdWU2I22S70L1dp2c2emOz86MrAru+vkSeF9TD+GGEdio+ti1JlPTCqe1MWjGPBQTUh8rJHuKc/z2SWmH5ddPzk7FVgMoNurQB6FeqXOnSS9C1NHFxPPeZbbQ93Yh16va/4wd/8lGN1lmOq6nKDfaxtypbkWZ/eUaI6TQtvPmjjuCZFYv7UvITpsTqwT8k7mloKdcOOil+ln/DvUPOLhUqJcoYS7xduaWaV0kkAmxWX4a3sgg+3tOWyK12A6ZdzAqCqbr7WrfkCJ18mC2GgVaekugBovStybQ6BDT68bsspW3OoNChpq5IdZUzg/U4ZP7mG4oqgfhqNp0nbd4mQOxBr8PAqvemsF62qUOXzROP3SlTqqnTQZYU8+lU+c/c1nsQX6oF92oYp/IQ+Up/Km3DYzcntidXzPrS/KUp5Y8GdC33etiS7H/dbPJ2C4qZe4UPTgbSgxAT3L+9uLyO6Aopn0qDiT0Br+GBpWXx+vhVQ6FuMXAzzqDDR4HKizGhxxzpZ1DIQGytx/OLa5lrqlAQbt6hKwLFZ4XNYJAwY6cbnENzdqKhcqOxuRp/n1WdiqXSxMuWmDRycufZLJ93llW+cUOMjXjjaB1LORrk6fO9FFlvQUGKqcoQGu8M3RdvUpxEX3rILtqBzUTnyDGGEmZDYdQzOLSfVYZHO8rrYTJkOssgFKfKyyIkKknxu3IXrA/UbWtFy3n3zOctwbp8HAB6/tWM79eIRJpYNxoqQKru89HLPL5ggxvxwY1uzMm+zOXpa33OUZIH6R5Eypa0x7EQvjvyPpoIefD9kZjUZSYBIMnKCzEFMw05asVfhdAL9QdkOZ9ZRkpkfVcTZiucZpGb8kTVtuPny6PaDeOJJZEY/iN9QxUVdg9OZ+k6EVM77yA6aXk2aDOfBrGlbB20/7m+M6gxFsa/QSUes6yOg7ab1t/yqFhFxtCE5bZSv5x+IX/MWjUmiQH+3viDk7fX8FZWeBepwCwtHdmGVnzgB6G8k72OHLUGhVagk8VEhYODaX9ACRTN3wBHXyVFNolIDfpN8Vctu2kf0Cuc9OEad2RSJVCasAAckoe2iITyItS0fbky/rDsPWQqSVErYwIRWZwvTPHwxILqeG7TNx0O0z9rVmlM7HmIIOj9j2jLb14TuZrA0knH+FDi75HNuAhgyD+XMelb5IzvDPwQt+FVE8bUdKIwIZ5h+X4pPBHtciNNwygF8ygfZ3jp/qiDDXPGGWsPu1OeuyFll4ao+yIShF7YRCgBYu7kQrFsu7CHK2VUIp8dWe0PdZiz/dzz3OM0lGgLfHx2cYrJi1JrS4m8i0iHqSrKRhDsIcsoG6QOGm4nsTjb7DL6VefL1+LtZN2iCVoVyjL7IiVb2hJjvd+upm/0WNfHtYBur+RgP0hM5hYOxV4TLD/nzKlzPwz00X47FwcIMaWaFx4bVF+NO/0k64qjTFOOtzxOKPOC8yRN5S2UTUbRyQ1NGqCx+MV3qmlpkrOKgl44F1+SrHFD96uV3Q0XBl6oCP9BuzroRg4nM5KHXu9oPaJE5H/t9MyDuww20pYPeBcmgt/0om49N87u3ke2P+u1iD33yYuLtAmWWYVlZjKoXw9ag2HY9QE1AgwrfRB4l9Cm8si7Bv49HAQg8RvQMRcKzscOGCqTuco896bdGxzCrZMf1oPQ+NIvYyPnKuz2pbk/gE15iJfTTyHvEHQRGBtCRWyWvpzMs5KZB/Oy8ARZdsHPbJPrDnIBpCjxeQKA7s0UFNDG18rY+/aR8hSfe6aVhjMiIR342CZgJUM2aATPAskAy6InoNmQUrVLjT3rDIrnIJJHYBZATUigHfQJZL960nER+FJJOnG2Hr8ofk4ol+5VHUklqjNRqFIweSwLfb3F50w5OjXkxuZBbRy3COj1aU4I6VAtnMk0L0sU/3sxNUEQqMfScxdZGBCdK4UkpiKU67Ww9WUwbI7a/F2mS5yHdY3ngY77la2AOifWqpmcdmz0PmDRYSBKMa2PEBR/Syv5EsR5w3yarKZvWvdmuPK25jxEJBvmjZP8y+a9FmVOt5lPtAAoM6hfDyLDztcsilQX86Ziog4AI+Jf0KJm5eRaTXV/6TR4YY5Pqw9t1Vsgcy/2kEEkUk/baN6QQjP15hN4hkUusIIrhm3BI72BzPxcd13iVFxjbAixeNyXOp6kvAK6vQ0XslNEn2/K8Q067EXMuCX3uwnBwIOyCLNBdP6s0MiaXDiWlFnJAC7k29c1CGozkAoGyFrDXGFhnoOUtH54zAz77n/0YyXWX342MtVCgeKYpR/sFGkmcvCWC4uWIzmly9Av66hbnNB4qRSlCXkPpc7hc85Af8JoBlaobUTWF4qYRMGQSz48JqCr0a59xe4OOhNa8kLfB/dAZrhC8qcdpRlG3TwCn93pTLXqreVcLedPFCuecojxsRx/3Jhmb6puGQMjhFcdB12DuoYPfQoD6Lzqj+OiWgTEzkhd+Sv5xJ1bcuP9IdfcpfKIIlzpG0RJmZOv8GTc1yeESS/+BqPUwge3sT3Kx11hk2t3iOR24e4+trdEr7SfpiLV4T+sYlNudl/JPzMqEmEX+JzxC9sbkQXO/IYDY4WDcaON7bJ7+V3exu59cmF4i+Vbk/ADRP7tL/7iNWNTO182lbzLGJvAUbtBSoj8Wsa629ODOvAfNNYXkvGgT5XtVyOIupGEhgpglnedbRRVquxbmLrJ57Vc0XA97gqEjiwmy+fsrM7Y8TmRBrhV5nXZ21ZNPOVeVRKNedw852ZJl1+wuDyxLDhEzY04LR9PELprLG3CC2vCzrd+B4yR+1rSYm8/RaM8XU8qCbtCVUNIGhaNChpgRsyQ6liiH9nrrKjaIkawQqiUWo4jEA6EzkaYXt313HuPPFpSrnoxD45aipP1f9+CsgnAUuTrhLP0DZMNZBMIctnwAM0aX8R4sZNe2359awZqPm/Epz928RXQC4ZCnPqhX6XWRuYfWxZXaUoIQ9Mv47+5kMGIR0GThmLRcb1OJETfSS3/MfCCeWpAc1misy+WvBYLQAvEi93tsl8E2Vitnykg0LNYiRy9FCPjDF6FG6pF2fVrXiE36dwgkAVxciTEi0TFOqpQRi+pxsDK+nGiyC0GQ03bhXXRwsHPbGG6dFGqs1GoIB+0P14oCR5uInJZyU0ZSOJvbpq4TRMyNTMCU5AC2ljEepwc4TfF/kr3HwPCoLoF77EXf3FHcUhp1bgw6VJ5bEA2AVsBj4zT4eWqcmfQfPqhSvAxaScJTzz8oOeq8oYFwp61P8rGZ07tIIcPZwNj0pBhszrSh0RlXWT6hLsZxnltvHcxmsf6A0o01fThKAARQQH/qE6KT1w9UQSq7mcxNSdtHMguAzdXsJaVUDyK4wA59LcRmLFU9j4RcWDJH4GVl0jiZuL1/kMyvk5A2G90G62NGHKn49z58NIA5GYvfPd5OCAZsRlx8zkOH1vyC3z/guYtO7Ysdje63ZiN75KBKyLDNoYFxbY7Zm/60e8yPQWfL0xRD8eKfjSepHIzBu/PietwmXxEocXMYZ6lLkQRs3HJWCSu+NbFXKJlsdYfBVv3gKThmp6BeW8STBQ1xeNcvKcJujO6o/9Rv2ArgL7GMzXwSLxM5//Lv/a3f266vybUOhSPmYADV6HAbd3jVSjGKKqM1Yjt2OT7GIWxg8sJLknJP4N1BMYp8qACBjW/hbr/T95DQRX5py4QP0zKJw1gieZgl+VZG7xyNrk/bO3uw3lTx9UjWsp5Ofnr9/552beB2dQ+gLskH5I7Udmsi3r72zRKfODDj/Kn1NPchx933fnqirFfXTIEv0NI6EJqTQMrbnsPYi7W5U+y8R3D9lFPbypazpIjrIUiMYcOctxspbCHBSdlUSUTi6Tn3zDKLZ3mBTL9aGUA8kjAO2qZLrFFHqY60SgcTZUlQOWZaYbPz0CIji4BBYlMX7MGgQaWere8iDLDkD9b2k1TmqMgMcdDnWI0w/akNJ3XN2OLgXL3PY5GsBV/AT/8CzYj4IqEz6+FACf1CeS+NYl0Mq9hgcZismpS9+eh4njCOYdb33N8D3UFDkGjQvs24tCDAQReiCB5WXnBrie7RgpvMU5JjHRoIFRmKdEvMSl8oRSeAP6dcAtItagXajuqx+f1P+83ehoSUdCjBkJK8rzRz3R76cZzWkM8wYqOhkCbZMo5FI439oWPSsKU0H6hcXbOBeD7NyvP/JGbZbAPkVzLTb7mf4W5Ylyf1OHFT3HcNK1FEFOSQoODeH8l7rFIfxd5k+RXC8fn3VRAzzyejBGuCRUQZI7xBFdxzfFIfJ1zQQO1qht0f9uZybvJOerA6Jo0Fj5yZIW1XVHebcbIHylSjJ/Mxcpt7aT/6Je4jOCNlOPDRzoveLI7UBZ8opGVVtmi+b6ETsrrs3QLEHrIMtzkK6G+zhOV4Wm5y6lNgTafPfbEeGG/DuxuPVehQ2Mp5QnDyBVEcEBw0+WvxHL45gsjkbT4EvcyyKEVlLzfcSh3SYVVX7tZLEQ8kq37JStXPgWcIt6Masm7E1/c9jQBs1+kQAKnJ1HxvkDDZICV1wF/b1/C2DOCa4ahKtjSvnRGuNKhMCt3NCF5yzRFbzAyIvA8AUrhm+3PVoMqgCDulHPk5G+WQVyZ+U3RuMnl+NNin6xwwq4blaf2w5c3Ce2G7vRGxBlqjv5hzj2QOPhd39ttpmLehgCfta1cz3dtOGQf8UGLC61yvgV1Rzjf6JyBXwOeAfM/WswP1tLZ1HUrFuXmIsfn2mB0diYmTLkdk2ZO88xsaOSvMs99OaeegTzecTO5/9uapWHBiKAcjUahdrGQo18TqyJQmNKABQH3sZZqj3FxRieWW3099pZK58C8u8luFkecy/Ijf2fFvtBBU34fnTBqRAH/8vKRpvuYO8BWV04xBqdPs6cSf7cGJBBYawUUY0bP+Q7Lz2YklyLiGVpGqBNRcBsSyHZzrpGihf3MYli9/UPN9aOLOF",
    "__VIEWSTATEGENERATOR": "11E838D2",
    "__EVENTVALIDATION": "zb+HN2+dsdkJQ3yodiHE4e1ZYx8eWNS1sBh+dxuoR249kr45X2YzV6gWBGoG5sPsPxrtQyalyQDj6NJhHmz1E3wGNkgAs6/PAtzrYbP2V59w1Ex9zlT7B1L7SWYoVw92FH7prPDm6BM2dD9OxQw6u4IgfrgT9e7tUuNzRJPqIn/iNd9HNejzkt1UMQky5BYdxUrRsp905ohnVjDfkoMffrSaZmXDG+NteJgJ7jfIJ+5/XMwTz+kFaOjnFk5V7VTJk3dJWOn9DGMrfRDNQJnPgY2lxvBz7SzKnf3lLO4gzJqsbvsUHpy23iouaUyaBUEsSMqnlDU8F2EUOL26+3dH4NaGW6LOZ1Bzz7EVix+4wfRmSg+/PSgcnJocDDZ/3bZgPJEOeOesJD8hrEseFtcVBXwjE9ae9vn0O9JptD3hlViPECD/+Tf/+Y18x9FcovcEgJ5BaBRGceCF7SrhwI0h7IwU97Hg7Z1KqIc36w3fiqEJpSJXVvjDJTRLWKT8A74zX8EbHKfaLmeukN8pzpb4+jIdgVMN/eMTGr4YgAEoX2BL5kw51KKpMQT2Y1F0WWUGtp/O63vHNFwv3bh7yW2avqLpNGT7ufc7dXVm8+zbsgp9ME84BdJLnr/0nref2UNA08/CIg==",
    "forma": "請選擇遊戲",
    "Lotto649Control_history$txtNO": "", # 配入空字串
    "Lotto649Control_history$chk": "radYM",
    "Lotto649Control_history$dropYear": 108,
    "Lotto649Control_history$dropMonth": 3,
    "Lotto649Control_history$btnSubmit": "查詢"
}

res = requests.post(url,headers=headers,data=payload)  ### data 為要攜帶的參數
# print(res.text)
soup = bs(res.text,"lxml")
# print(soup)

In [2]:
### 整理解析資料的 function 
def get_clear_data(soup):
    tables = soup.select("table#Lotto649Control_history_dlQuery table")
    
    result = []
    
    for table_tag in tables:

        # 期別 , 開獎日 -> 第二個 tr
        tr2 = table_tag.select("tr")[1]

        lotto_no   = tr2.select("td")[0].text
        lotto_date = tr2.select("td")[1].text.strip()

        # 樂透號碼 -> 第五個 tr
        tr5 = table_tag.select("tr")[4]

        # List comprehension
        lotto_numbers_short = [ td_tag.text.strip() for td_tag in tr5.select("td")[1:] ]
        
        result.append({
            "no"   : lotto_no,
            "time" : lotto_date,
            "numbers" : lotto_numbers_short
        })

    return result

get_clear_data(soup)

[{'no': '108000033',
  'time': '108/03/29',
  'numbers': ['01', '03', '12', '16', '29', '49', '04']},
 {'no': '108000032',
  'time': '108/03/26',
  'numbers': ['06', '13', '14', '19', '24', '38', '26']},
 {'no': '108000031',
  'time': '108/03/22',
  'numbers': ['01', '12', '16', '37', '41', '47', '36']},
 {'no': '108000030',
  'time': '108/03/19',
  'numbers': ['07', '16', '17', '36', '47', '48', '49']},
 {'no': '108000029',
  'time': '108/03/15',
  'numbers': ['05', '10', '12', '20', '28', '36', '15']},
 {'no': '108000028',
  'time': '108/03/12',
  'numbers': ['05', '07', '22', '37', '40', '45', '12']},
 {'no': '108000027',
  'time': '108/03/08',
  'numbers': ['03', '11', '15', '25', '31', '48', '36']},
 {'no': '108000026',
  'time': '108/03/05',
  'numbers': ['12', '13', '16', '21', '42', '43', '39']},
 {'no': '108000025',
  'time': '108/03/01',
  'numbers': ['01', '08', '14', '27', '40', '45', '34']}]

In [12]:
### 取得隱藏參數
def get_hidden_params(soup):
    __VIEWSTATE = soup.select("input#__VIEWSTATE")[0]["value"]
    __VIEWSTATEGENERATOR = soup.select("input#__VIEWSTATEGENERATOR")[0]["value"]
    __EVENTVALIDATION = soup.select("input#__EVENTVALIDATION")[0]["value"]
    
    return {
        "__VIEWSTATE" : __VIEWSTATE,
        "__VIEWSTATEGENERATOR" : __VIEWSTATEGENERATOR,
        "__EVENTVALIDATION" : __EVENTVALIDATION
    }


# get_hidden_params(soup)

In [7]:
### 抓取 1 ~ 5 月 樂透號碼

url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

output = []

### 發 POST requests 取得頁面 source code
for i in range(1,6):
    hidden = get_hidden_params(soup)
    payload = {
        "__VIEWSTATE": hidden["__VIEWSTATE"],
        "__VIEWSTATEGENERATOR": hidden["__VIEWSTATEGENERATOR"],
        "__EVENTVALIDATION": hidden["__EVENTVALIDATION"],
        "forma": "請選擇遊戲",
        "Lotto649Control_history$txtNO": "", # 配入空字串
        "Lotto649Control_history$chk": "radYM",
        "Lotto649Control_history$dropYear": 108,
        "Lotto649Control_history$dropMonth": i,
        "Lotto649Control_history$btnSubmit": "查詢"
    }

    res = requests.post(url,headers=headers,data=payload)  ### data 為要攜帶的參數
    soup = bs(res.text,"lxml")
    
    data = get_clear_data(soup)
    output += data
    
    # print(data)
    print("{}月 is ok.".format(i))
    print("-"*80)
    
print("Done.")

1月 is ok.
--------------------------------------------------------------------------------
2月 is ok.
--------------------------------------------------------------------------------
3月 is ok.
--------------------------------------------------------------------------------
4月 is ok.
--------------------------------------------------------------------------------
5月 is ok.
--------------------------------------------------------------------------------
Done.


In [17]:
### Sorted 排序 function
# sorted(output , key = lambda ele : ele["time"])

In [6]:
### list 疊加
a = [1,2,3]
b = [4,5,6]

print(a+b)

c = [0,0,0]
c += a
print(c)

[1, 2, 3, 4, 5, 6]
[0, 0, 0, 1, 2, 3]


In [18]:
### 組合程式

## 引用套件
import requests
from bs4 import BeautifulSoup as bs

## 定義 function

def get_hidden_params(soup):
    __VIEWSTATE = soup.select("input#__VIEWSTATE")[0]["value"]
    __VIEWSTATEGENERATOR = soup.select("input#__VIEWSTATEGENERATOR")[0]["value"]
    __EVENTVALIDATION = soup.select("input#__EVENTVALIDATION")[0]["value"]
    
    return {
        "__VIEWSTATE" : __VIEWSTATE,
        "__VIEWSTATEGENERATOR" : __VIEWSTATEGENERATOR,
        "__EVENTVALIDATION" : __EVENTVALIDATION
    }

def get_clear_data(soup):
    tables = soup.select("table#Lotto649Control_history_dlQuery table")
    
    result = []
    
    for table_tag in tables:

        # 期別 , 開獎日 -> 第二個 tr
        tr2 = table_tag.select("tr")[1]

        lotto_no   = tr2.select("td")[0].text
        lotto_date = tr2.select("td")[1].text.strip()

        # 樂透號碼 -> 第五個 tr
        tr5 = table_tag.select("tr")[4]

        # List comprehension
        lotto_numbers_short = [ td_tag.text.strip() for td_tag in tr5.select("td")[1:] ]
        
        result.append({
            "no"   : lotto_no,
            "time" : lotto_date,
            "numbers" : lotto_numbers_short
        })

    return result


## main 程式
url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

# 1. 發 GET requests , 取得第一次的 隱藏參數
res = requests.get(url,headers=headers) 
soup = bs(res.text,"lxml")
# hidden = get_hidden_params(soup)
# print(hidden)


# 2. for-loop 發 POST requests , 取得 1 ~ 6 月的樂透號碼
output = []

for i in range(1,7):
    hidden = get_hidden_params(soup)
    payload = {
        "__VIEWSTATE": hidden["__VIEWSTATE"],
        "__VIEWSTATEGENERATOR": hidden["__VIEWSTATEGENERATOR"],
        "__EVENTVALIDATION": hidden["__EVENTVALIDATION"],
        "forma": "請選擇遊戲",
        "Lotto649Control_history$txtNO": "", # 配入空字串
        "Lotto649Control_history$chk": "radYM",
        "Lotto649Control_history$dropYear": 109,
        "Lotto649Control_history$dropMonth": i,
        "Lotto649Control_history$btnSubmit": "查詢"
    }

    res = requests.post(url,headers=headers,data=payload)  ### data 為要攜帶的參數
    soup = bs(res.text,"lxml")
    
    data = get_clear_data(soup)
    output += data

    print("{}月 is ok.".format(i))
    
print("Done.")

# sorted(output , key = lambda ele : ele["no"])

1月 is ok.
2月 is ok.
3月 is ok.
4月 is ok.
5月 is ok.
6月 is ok.
Done.


In [ ]:
##########################################################################################

In [ ]:
##########################################################################################

In [ ]:
##########################################################################################

In [40]:
##### PTT 股票版 （重新開始爬蟲）

### 取得首頁 source code
import requests
from bs4 import BeautifulSoup as bs 

url = "https://www.ptt.cc/bbs/Stock/index.html"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")
# soup

In [28]:
### 抓取首頁文章連結
# soup.select("div#main-container div.r-ent a")  # 不佳

links = []
for a_tag in soup.select("div#main-container div.r-ent div.title a"):
    
    # 過濾 版規 & 盤後閒聊 / 盤中閒聊
    title = a_tag.text
    
    if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
        continue # 跳過此步, 執行下一動迴圈
    else:
        url = "https://www.ptt.cc" + a_tag["href"]
        print(url,title)

https://www.ptt.cc/bbs/Stock/M.1636020592.A.367.html [請益] 為何美國主動基金長期績效很少能打贏VTI
https://www.ptt.cc/bbs/Stock/M.1636020879.A.366.html [新聞] 〈聯詠法說〉明年全產品線出貨續強 首顆O
https://www.ptt.cc/bbs/Stock/M.1636021794.A.AD5.html [新聞] 陽明海運捐贈279台空氣清淨機
https://www.ptt.cc/bbs/Stock/M.1636021830.A.68E.html [新聞] 華邦電前3季獲利增8.8倍 每股賺2.36元
https://www.ptt.cc/bbs/Stock/M.1636023729.A.61E.html [標的]2476鉅祥
https://www.ptt.cc/bbs/Stock/M.1636024843.A.C1B.html [新聞] 高通Q4展望優於聯發科 後續觀察市占消長 
https://www.ptt.cc/bbs/Stock/M.1636025058.A.7D6.html [新聞] 《航運股》台驊投控Q3本業創高 前3季每ꨮ


In [41]:
### 抓取 分頁文章 連結
links = []

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

for i in range(1,6):
    
    # 建構 '上頁' 連結
    link = soup.select("div#action-bar-container div.btn-group-paging a")[1]["href"]
    previous_link = "https://www.ptt.cc" + link

    res = requests.get(previous_link,headers=headers)
    soup = bs(res.text,"lxml")

    for a_tag in soup.select("div#main-container div.r-ent div.title a"):

        # 過濾 版規 & 盤後閒聊 / 盤中閒聊
        title = a_tag.text

        if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
            continue # 跳過此步, 執行下一動迴圈
        else:
            url = "https://www.ptt.cc" + a_tag["href"]
            links.append(url)
            
    print("{} is ok.".format(previous_link))
        
print("Done.")

https://www.ptt.cc/bbs/Stock/index5018.html is ok.
https://www.ptt.cc/bbs/Stock/index5017.html is ok.
https://www.ptt.cc/bbs/Stock/index5016.html is ok.
https://www.ptt.cc/bbs/Stock/index5015.html is ok.
https://www.ptt.cc/bbs/Stock/index5014.html is ok.
Done.


In [76]:
# links

In [110]:
### 抓取文章本文 source code 
url = links[17]
print(url)

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res2 = requests.get(url,headers=headers)
soup2 = bs(res2.text,"lxml")

# print(soup2)

https://www.ptt.cc/bbs/Stock/M.1636004790.A.729.html


In [111]:
### 抓取本文的 作者 , 看板 , 標題 , 時間 
span_tags = soup2.select("div#main-content span.article-meta-value")

# 作者
author = span_tags[0].text
print("author :",author)

# 看板
category = span_tags[1].text
print("category :",category)

# 標題
title = span_tags[2].text
print("title :",title)

# 時間
time = span_tags[3].text
print("time :",time)

# span_tags

author : what5566 (打我阿笨蛋)
category : Stock
title : [請益] 台股現在是世界最爛嗎？
time : Thu Nov  4 13:46:28 2021


In [112]:
# print(soup2.select("div#main-content")[0].text)

In [113]:
### 抓取本文的 內容 , 回應
# 標籤拔除 --> .extract()
# 為了標籤拔除 , 故先抓取 回應資料

# push_tags = soup2.select("div#main-content div.push")

# 抓其中一項做測試
# ele = push_tags[77]
# span_tags = ele.select("span")

# resp = {
#     "tag"     : span_tags[0].text.strip(),
#     "author"  : span_tags[1].text.strip(),
#     "content" : span_tags[2].text.replace(": ","").strip(), 
#     "time"    : span_tags[3].text.strip()
# }
# print(resp)
# span_tags

#############
# for-loop 處理全部回應
# resp_data = []

# for ele in push_tags:
#     span_tags = ele.select("span")

#     resp = {
#         "tag"     : span_tags[0].text.strip(),
#         "author"  : span_tags[1].text.strip(),
#         "content" : span_tags[2].text.replace(": ","").strip(), 
#         "time"    : span_tags[3].text.strip()
#     }
    
#     resp_data.append(resp)
    
#############
# 加入拔除標籤動作
def get_resp_data(ele):
    span_tags = ele.select("span")
    return {
        "tag"     : span_tags[0].text.strip(),
        "author"  : span_tags[1].text.strip(),
        "content" : span_tags[2].text.replace(": ","").strip(), 
        "time"    : span_tags[3].text.strip()
    }


push_tags = soup2.select("div#main-content div.push")
resp_data = []

if len(push_tags) >0:
    
    for ele in push_tags:
        ele.extract()  # 宣告從 div#main-content 中,拔除 div.push 標籤
        
        resp = get_resp_data(ele)
        
        resp_data.append(resp)
        
#         span_tags = ele.select("span")
        
#         resp = {
#             "tag"     : span_tags[0].text.strip(),
#             "author"  : span_tags[1].text.strip(),
#             "content" : span_tags[2].text.replace(": ","").strip(), 
#             "time"    : span_tags[3].text.strip()
#         }

    
print(resp_data)

[{'tag': '噓', 'author': 'yanchi030', 'content': '不是你嗎', 'time': '11/04 13:47'}, {'tag': '推', 'author': 'au677777', 'content': 'https://i.imgur.com/EXVsUbn.jpg', 'time': '11/04 13:47'}, {'tag': '推', 'author': 'Enzofulgura', 'content': '4', 'time': '11/04 13:47'}, {'tag': '→', 'author': 'stocktonty', 'content': '不是啊 離前高不到5%是在爛什麼 沒遇過空頭逆', 'time': '11/04 13:47'}, {'tag': '推', 'author': 'ayiela', 'content': '真的爛', 'time': '11/04 13:47'}, {'tag': '推', 'author': 'OOorc', 'content': '先蹲後跳。台股', 'time': '11/04 13:47'}, {'tag': '→', 'author': 'pimday', 'content': '最近真的有人玩到賠錢?', 'time': '11/04 13:48'}, {'tag': '推', 'author': 'spyget', 'content': '4  最後一盤還被出貨  真滴爛', 'time': '11/04 13:48'}, {'tag': '噓', 'author': 'HERJORDAN', 'content': '昨天亞股全綠，只有台股紅你知道嗎？', 'time': '11/04 13:48'}, {'tag': '噓', 'author': 'jason748', 'content': '先前的大多頭時代都當不存在？', 'time': '11/04 13:48'}, {'tag': '推', 'author': 'varbutterfly', 'content': '是，傷心~', 'time': '11/04 13:48'}, {'tag': '噓', 'author': 'aaaRengar', 'content': '444

In [114]:
# 內容
# print(soup2.select("div#main-content")[0].text)

# 1. 移除標籤
# - div.article-metaline
# - div.article-metaline-right
# - span.f2

def remove_dirty_tag(soup):
    
    # 若存在 , 則移除標籤
    if len(soup.select("div.article-metaline")) >0 :
        
        # 標籤可能多項 , 使用 for-loop 移除
        for tag in soup.select("div.article-metaline"):
            tag.extract()
            
    if len(soup.select("div.article-metaline-right")) >0 :
        for tag in soup.select("div.article-metaline-right"):
            tag.extract()
            
    if len(soup.select("span.f2")) >0 :
        for tag in soup.select("span.f2"):
            tag.extract()
    
    return soup 



soup2 = remove_dirty_tag(soup2)
# print(soup2.select("div#main-content")[0].text.strip())

content = soup2.select("div#main-content")[0].text.strip()
print("content:", content)

content: 美股為我們上演了永動機

每天噴噴噴

基本上目前已知利空都反映過了

縮債 升息 限電 恆大

費半一個月漲10%
標普 納斯達克 道瓊一個月內也都狂噴 創高

昨晚FED出來講話後從平盤硬是拉了1%上去

日本 韓國 香港 中國 全部的指數都是紅的

結果台股在櫃買連續噴十幾天後

依舊在17000徘徊 甚至今天還是綠的

昨晚一片看好 今天站上17200劍指17500似乎只是一場夢

記得幾個月前的推文永遠是台股噴 台股噴完美股噴 美股噴完台股噴的循環

現在只剩台股在跌

台股是世界最爛嗎

--
